In [42]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework, ArgumentationSettings

from src.explainer.adjective import BooleanAdjective, PointerAdjective, ComparisonAdjective, MaxRankAdjective, MinRankAdjective
from src.explainer.explanation import Possession, Assumption, PossessionCondition, ConditionalExplanation, CompositeExplanation

In [43]:
# Example usage:
class MinMaxNode:
    def __init__(self, id, *, score=None, maximizing_player_turn=True, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.children = children or []
        self.parent = None
        self.maximizing_player_turn = maximizing_player_turn
        
        self.is_leaf = True
        if len(self.children) > 0:
            self.is_leaf = False
            for child in children:
                child.parent=self
                child.maximizing_player_turn = not self.maximizing_player_turn
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf11 = MinMaxNode('leaf11', score=3)
leaf12 = MinMaxNode('leaf12', score=4)
leaf21 = MinMaxNode('leaf21', score=8)
leaf22 = MinMaxNode('leaf22', score=2)
child1 = MinMaxNode('child1', children=[leaf11, leaf12], score_child=leaf11)
child2 = MinMaxNode('child2', children=[leaf21, leaf22], score_child=leaf22)
root = MinMaxNode('root', maximizing_player_turn=True, children=[child1, child2], score_child=child1)

In [44]:
knowledgebase = ArgumentationFramework()

knowledgebase.add_adjective( 
    BooleanAdjective("leaf",
        definition = "node.is_leaf")
)

knowledgebase.add_adjective( 
    PointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = PossessionCondition("leaf"),
            true_explanation = Assumption("Leaf nodes have scores from the evaluation function"),
            false_explanation = CompositeExplanation(
                Assumption("Internal nodes have scores from children"),
                Possession("backtracing child"))
        ))
)

knowledgebase.add_adjective(
    BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn")
)

knowledgebase.add_adjective( 
    PointerAdjective("backtracing child",
        definition = "node.score_child",
        # alternative definition for backtracing child:
        # min(node.children, key=lambda child: child.score),

        explanation = ConditionalExplanation(
            condition = PossessionCondition("opponent player turn"),
            true_explanation = CompositeExplanation(
                Assumption("We assume the opponent will do their best move."),
                Possession("backtracing child", "worst")),
            false_explanation = CompositeExplanation(
                Assumption("On our turn we take the maximum rated move."),
                Possession("backtracing child", "best"))
        ))
)

knowledgebase.add_adjective(
    ComparisonAdjective("better", "score", ">")
)

knowledgebase.add_adjective( 
    PointerAdjective("siblings",
        definition = "[sibling for sibling in node.parent.children if sibling is not node]")
)

knowledgebase.add_adjective(
    MaxRankAdjective("best", "better", "siblings")
)
knowledgebase.add_adjective(
    MinRankAdjective("worst", "better", "siblings")
)

In [45]:
print(knowledgebase)

Propositions:
node is leaf
node has score = ?
node is opponent player turn
node has backtracing child = ?
node1 is better than node2
node has siblings = ?
node is best
node is worst


Implications:
[(assumption) Definition of "leaf" is node.is_leaf → node is leaf]

[[(assumption) Definition of "leaf" is node.is_leaf → node is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function
 ∨ [(assumption) Definition of "leaf" is node.is_leaf → ¬(node is leaf)]
 ∧ (assumption) Internal nodes have scores from children ∧ [[(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn → node is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move. ∧ [(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ Node ¬(better) than all nodes in siblings → node is worst]

 ∨ [(assumption) Definition of "opponent player turn" is not node.maximizing_player_turn → ¬(node is opponent play

In [46]:
explainer = ArgumentativeExplainer(knowledgebase)

explainer.set_tree_search_motivation(lambda root: root.children, "best")

In [47]:
settings = {
            'explanation_depth': 3 ,
            'assumptions_verbosity': 'no',
            'repeat_expl_each_node': True
        }

knowledgebase.configure_settings(settings)

In [48]:
# Generate explanations
print(explainer.explain_adjective(child1, "best"))

[[[¬(child1 is leaf) ∧ [child1 is opponent player turn ∧ [[[leaf11 is leaf → leaf11 has score = 3]
 ∧ [leaf12 is leaf → leaf12 has score = 4]
 → ¬(leaf11 is better than leaf12)]
 → leaf11 is worst]
 → child1 has backtracing child = leaf11]
 → child1 has score = 3]
 ∧ [¬(child2 is leaf) ∧ [child2 is opponent player turn ∧ [[[leaf22 is leaf → leaf22 has score = 2]
 ∧ [leaf21 is leaf → leaf21 has score = 8]
 → ¬(leaf22 is better than leaf21)]
 → leaf22 is worst]
 → child2 has backtracing child = leaf22]
 → child2 has score = 2]
 → child1 is better than child2]
 → child1 is best]



In [49]:
print(explainer.explain_adjective(child1, "best"))

[[[¬(child1 is leaf) ∧ [child1 is opponent player turn ∧ [[[leaf11 is leaf → leaf11 has score = 3]
 ∧ [leaf12 is leaf → leaf12 has score = 4]
 → ¬(leaf11 is better than leaf12)]
 → leaf11 is worst]
 → child1 has backtracing child = leaf11]
 → child1 has score = 3]
 ∧ [¬(child2 is leaf) ∧ [child2 is opponent player turn ∧ [[[leaf22 is leaf → leaf22 has score = 2]
 ∧ [leaf21 is leaf → leaf21 has score = 8]
 → ¬(leaf22 is better than leaf21)]
 → leaf22 is worst]
 → child2 has backtracing child = leaf22]
 → child2 has score = 2]
 → child1 is better than child2]
 → child1 is best]



In [50]:
knowledgebase.settings.explanation_depth = 1

In [51]:
print(explainer.explain_adjective(child1, "best"))

child1 is best


In [52]:
knowledgebase.settings.explanation_depth = 3

In [53]:
print(explainer.explain_adjective(child1, "worst"))

[[[¬(child1 is leaf) ∧ [child1 is opponent player turn ∧ [[[leaf11 is leaf → leaf11 has score = 3]
 ∧ [leaf12 is leaf → leaf12 has score = 4]
 → ¬(leaf11 is better than leaf12)]
 → leaf11 is worst]
 → child1 has backtracing child = leaf11]
 → child1 has score = 3]
 ∧ [¬(child2 is leaf) ∧ [child2 is opponent player turn ∧ [[[leaf22 is leaf → leaf22 has score = 2]
 ∧ [leaf21 is leaf → leaf21 has score = 8]
 → ¬(leaf22 is better than leaf21)]
 → leaf22 is worst]
 → child2 has backtracing child = leaf22]
 → child2 has score = 2]
 → child1 is better than child2]
 → ¬(child1 is worst)]



In [54]:
print(explainer.explain_adjective(leaf11, "worst"))

[[[leaf11 is leaf → leaf11 has score = 3]
 ∧ [leaf12 is leaf → leaf12 has score = 4]
 → ¬(leaf11 is better than leaf12)]
 → leaf11 is worst]



In [55]:
print(explainer.explain_adjective(child2, "best"))

[[[¬(child2 is leaf) ∧ [child2 is opponent player turn ∧ [[[leaf22 is leaf → leaf22 has score = 2]
 ∧ [leaf21 is leaf → leaf21 has score = 8]
 → ¬(leaf22 is better than leaf21)]
 → leaf22 is worst]
 → child2 has backtracing child = leaf22]
 → child2 has score = 2]
 ∧ [¬(child1 is leaf) ∧ [child1 is opponent player turn ∧ [[[leaf11 is leaf → leaf11 has score = 3]
 ∧ [leaf12 is leaf → leaf12 has score = 4]
 → ¬(leaf11 is better than leaf12)]
 → leaf11 is worst]
 → child1 has backtracing child = leaf11]
 → child1 has score = 3]
 → ¬(child2 is better than child1)]
 → ¬(child2 is best)]



In [56]:
print(explainer.explain_adjective(root, "backtracing child"))

[¬(root is opponent player turn) ∧ [[[¬(child1 is leaf) ∧ [child1 is opponent player turn ∧ [[[leaf11 is leaf → leaf11 has score = 3]
 ∧ [leaf12 is leaf → leaf12 has score = 4]
 → ¬(leaf11 is better than leaf12)]
 → leaf11 is worst]
 → child1 has backtracing child = leaf11]
 → child1 has score = 3]
 ∧ [¬(child2 is leaf) ∧ [child2 is opponent player turn ∧ [[[leaf22 is leaf → leaf22 has score = 2]
 ∧ [leaf21 is leaf → leaf21 has score = 8]
 → ¬(leaf22 is better than leaf21)]
 → leaf22 is worst]
 → child2 has backtracing child = leaf22]
 → child2 has score = 2]
 → child1 is better than child2]
 → child1 is best]
 → root has backtracing child = child1]



In [57]:
print(explainer.explain_adjective(root, "score"))

[¬(root is leaf) ∧ [¬(root is opponent player turn) ∧ [[[¬(child1 is leaf) ∧ [child1 is opponent player turn ∧ [[[leaf11 is leaf → leaf11 has score = 3]
 ∧ [leaf12 is leaf → leaf12 has score = 4]
 → ¬(leaf11 is better than leaf12)]
 → leaf11 is worst]
 → child1 has backtracing child = leaf11]
 → child1 has score = 3]
 ∧ [¬(child2 is leaf) ∧ [child2 is opponent player turn ∧ [[[leaf22 is leaf → leaf22 has score = 2]
 ∧ [leaf21 is leaf → leaf21 has score = 8]
 → ¬(leaf22 is better than leaf21)]
 → leaf22 is worst]
 → child2 has backtracing child = leaf22]
 → child2 has score = 2]
 → child1 is better than child2]
 → child1 is best]
 → root has backtracing child = child1]
 → root has score = 3]



Track down that Not before the "Considering your definition of leaf"

In [58]:
print(explainer.query_explanation(root, "Why is child 1 maxoptimal?"))

None
